# Welcome to the Metaverse

To truely understand the workings of Python, you'll need to be unafraid to enter the Metaverse of Python Programming and get your hands dirty with a little metaprogramming.

What is metaprogramming?

_The term __metaprogramming__ refers to the potential for a program to have knowledge of or manipulate itself._ - https://realpython.com/python-metaclasses/

Where can I find metaprogramming?

In your day to day use you'll be suprised how much meta programming you're actually using. The best example is `@dataclass`.

```python
from dataclasses import dataclass

@dataclass
def DataObject:
    field_1: str
    field_2: int
```

When looking at this example you have to ask what exactly is happening here? How did my class definition which contains two class attribute definitions turn into an instantiable class? Without the `@dataclass` decorator, this class would not initialize with the expected inputs.

That's where the `@dataclass` decorator does some metaprogramming. It looks at your definition of the class at class definition time, reads the class attributes, and modifies/creates the constructor to create the equivalent...

```python
def DataObject:
    def __init__(self, field_1, field_2):
        self.field_1, self.field_2 = field_1, field_2
    ... # plus some free extras like __repr__, etc...
```


## A common method of metaprogramming is metaclasses

Metaclasses receive a class definition and can inspect, modify, remove, or act on it as it sees fit.

Let's look at a common metaclass example of a class that checks for a certain class attribute to exist in a class definition...

In [25]:
class FriendsCheckMeta(type):
    def __new__(meta, name, bases, class_dict):
        print(f'* Running {meta}.__new__ for {name}')
        print(f'Bases:', bases)
        print(class_dict)
        if bases:
            if 'friends' not in class_dict or class_dict['friends'] is None:
                raise ValueError("Your class needs 'friends' my dude")
        return type.__new__(meta, name, bases, class_dict)

In [26]:
class MyPersonClass(metaclass=FriendsCheckMeta):
    friends = None
    def say_hi(self):
        print(f"hi, my {self.friends} friends!")

* Running <class '__main__.FriendsCheckMeta'>.__new__ for MyPersonClass
Bases: ()
{'__module__': '__main__', '__qualname__': 'MyPersonClass', 'friends': None, 'say_hi': <function MyPersonClass.say_hi at 0x7fd3e839e310>}


In [27]:
class MyCorrectPersonClass(MyPersonClass):
    friends = 10

* Running <class '__main__.FriendsCheckMeta'>.__new__ for MyCorrectPersonClass
Bases: (<class '__main__.MyPersonClass'>,)
{'__module__': '__main__', '__qualname__': 'MyCorrectPersonClass', 'friends': 10}


In [28]:
class MyWrongClass(MyPersonClass):
    pals = 299

* Running <class '__main__.FriendsCheckMeta'>.__new__ for MyWrongClass
Bases: (<class '__main__.MyPersonClass'>,)
{'__module__': '__main__', '__qualname__': 'MyWrongClass', 'pals': 299}


ValueError: Your class needs 'friends' my dude

While this works, there's two main issues with it:
1. It's pretty complicated for a simple task, and not everyone is familiar with Metaclasses
1. Current metaclass implementation in Python limits you to only having one meta class per class definition. This really messes things up when you try multiple inheritance with another class that has a metaclass.

Bonus: If you do want composable metaclasses, you might actually just need class decorators (_Item 51: Prefer Class decorators over Metaclasses for Composable Class extensions_)

## The alternative for most of your metaclass usage

`__init_subclass__`

In [33]:
class MyBetterPersonClass:
    friends = None
    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.friends is None:
            raise ValueError("Your class needs 'friends' my dude")

    def say_hi(self):
        print(f"hi, my {self.friends} friends!")
        

In [36]:
class MyCorrectPersonClass(MyPersonClass):
    friends = 10

MyCorrectPersonClass().say_hi()

hi, my 10


In [34]:
class MyWrongClass(MyBetterPersonClass):
    pals = 299

ValueError: Your class needs 'friends' my dude

## Homework: How does a library like Pydantic support the `Field` functionality?

E.g: https://pydantic-docs.helpmanual.io/usage/schema/#field-customization

_*Hint it involves metaprogramming._ ;)

```python
from pydantic import BaseModel, Field
class Model(BaseModel):
    # Here both constraints will be applied and the schema
    # will be generated correctly
    foo: int = Field(..., gt=0, lt=10)
```